# How to work with Converse API in Amazon Bedrock - Getting Started.

*Note: This notebook has been adapted from the [Getting started with the Converse API in Amazon Bedrock](https://github.com/aws-samples/amazon-bedrock-samples/blob/main/introduction-to-bedrock/Getting_started_with_Converse_API.ipynb)*

### Overview

In this notebook, we'll explore the basics of the Converse API in Amazon Bedrock. The Converse or ConverseStream API is a unified structured text API action that allows you simplifying the invocations to Bedrock LLMs, using a universal syntax and message structured prompts for any of the supported model providers.

To use the Converse API, you call the `Converse` or `ConverseStream` operations to send messages to a model. To call Converse, you require permission for the `bedrock:InvokeModel` operation. To call ConverseStream, you require permission for the `bedrock:InvokeModelWithResponseStream` operation.

### Prerequisites

Before you can use Amazon Bedrock, you must carry out the following steps:

- Sign up for an AWS account (if you don't already have one) and IAM Role with the necessary permissions for Amazon Bedrock, see [AWS Account and IAM Role](https://docs.aws.amazon.com/bedrock/latest/userguide/getting-started.html#new-to-aws).
- Request access to the foundation models (FM) that you want to use, see [Request access to FMs](https://docs.aws.amazon.com/bedrock/latest/userguide/getting-started.html#getting-started-model-access). 
    
    We have used below Foundation Models in our examples in this Notebook in `us-west-2` (Oregon) region.
    
| Provider Name | Foundation Model Name | Model Id |
| ------- | ------------- | ------------- |
| Amazon | Titan Text G1 - Express  | amazon.titan-text-express-v1 |
| Amazon | Titan Text G1 - Lite | amazon.titan-text-lite-v1 |
| Anthropic | Claude 3.5 Sonnet  | anthropic.claude-3-5-sonnet-20240620-v1:0 |
| Anthropic | Claude 3 Haiku  | anthropic.claude-3-haiku-20240307-v1:0 |
| Cohere | Command R+ | cohere.command-r-plus-v1:0 |
| Cohere | Command R | cohere.command-r-v1:0 |
| Meta | Llama 3.1 70B Instruct | meta.llama3-1-70b-instruct-v1:0 |
| Meta | Llama 3.1 8B Instruct | meta.llama3-1-8b-instruct-v1:0 |
| Mistral AI | Mistral Large 2 (24.07) | mistral.mistral-large-2407-v1:0 |
| Mistral AI | Mixtral 8X7B Instruct | mistral.mixtral-8x7b-instruct-v0:1 |


### Setup

Run the cells in this section to install the packages needed by this notebook.

In [15]:
!pip install --upgrade --force-reinstall boto3

import boto3
import sys
print('Running boto3 version:', boto3.__version__)

  Using cached boto3-1.35.10-py3-none-any.whl.metadata (6.6 kB)
  Using cached botocore-1.35.10-py3-none-any.whl.metadata (5.7 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.10.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
Using cached boto3-1.35.10-py3-none-any.whl (139 kB)
Using cached botocore-1.35.10-py3-none-any.whl (12.5 MB)
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
Using cached s3transfer-0.10.2-py3-none-any.whl (82 kB)
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
Using cached urllib3-2.2.2-py3-none-any.whl (121 kB)
Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.2
    Uninstalling urllib3-2.2.2:
      Successfully

Let's define the region and models to use. We can also setup our boto3 client.

In [16]:
region = 'us-west-2'
print('Using region: ', region)

bedrock = boto3.client(
    service_name = 'bedrock-runtime',
    region_name = region,
    )

MODEL_IDS = [
    "amazon.titan-text-express-v1",
    "amazon.titan-text-lite-v1",
    "anthropic.claude-3-5-sonnet-20240620-v1:0",
    "anthropic.claude-3-haiku-20240307-v1:0",
    "cohere.command-r-plus-v1:0",
    "cohere.command-r-v1:0",
    "meta.llama3-1-70b-instruct-v1:0",
    "meta.llama3-1-8b-instruct-v1:0",
    "mistral.mistral-large-2407-v1:0",
    "mistral.mixtral-8x7b-instruct-v0:1"
    ]


Using region:  us-west-2


We're now ready to setup our Converse API action in Bedrock. Note that we use the same syntax for any model, including the messages-formatted prompts, and the inference parameters. Also note that we read the output in the same way independently of the model used.

Optionally, we could define additional model specific request fields that are not common across all providers. For more information on this check the Bedrock Converse API documentation.

### Converse for one-shot invocations

In [17]:
def invoke_bedrock_model(client, id, prompt, max_tokens=2000, temperature=0, top_p=0.9):
    response = ""
    try:
        response = client.converse(
            modelId=id,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "text": prompt
                        }
                    ]
                }
            ],
            inferenceConfig={
                "temperature": temperature,
                "maxTokens": max_tokens,
                "topP": top_p
            }
            #additionalModelRequestFields={
            #}
        )
    except Exception as e:
        print(e)
        result = "Model invocation error"
    try:
        result = response['output']['message']['content'][0]['text'] \
        + '\n--- Latency: ' + str(response['metrics']['latencyMs']) \
        + 'ms - Input tokens:' + str(response['usage']['inputTokens']) \
        + ' - Output tokens:' + str(response['usage']['outputTokens']) + ' ---\n'
        return result
    except Exception as e:
        print(e)
        result = "Output parsing error"
    return result

Finally, we can test our invocation.

In this example, we run the same prompt across all the text models supported in Bedrock by the time of writing this example.

In [18]:
prompt = ("What is the capital of Italy?")
print(f'Prompt: {prompt}\n')

for i in MODEL_IDS:
    response = invoke_bedrock_model(bedrock, i, prompt)
    print(f'Model: {i}\n{response}')

Prompt: What is the capital of Italy?

Model: amazon.titan-text-express-v1
The capital of Italy is Rome. It is the fourth most populous city in the European Union.
--- Latency: 1258ms - Input tokens:10 - Output tokens:24 ---

Model: amazon.titan-text-lite-v1
The capital of Italy is Rome. Rome is the capital of both the country of Italy and the city of Rome.
--- Latency: 1154ms - Input tokens:10 - Output tokens:27 ---

Model: anthropic.claude-3-5-sonnet-20240620-v1:0
The capital of Italy is Rome.

Rome, also known as Roma in Italian, is not only the capital city but also the largest city in Italy. It has a rich history spanning over 2,500 years and is often referred to as the "Eternal City." Some key facts about Rome include:

1. It's located in the central-western portion of the Italian peninsula.

2. It serves as the seat of the Italian government and is home to many important national institutions.

3. Vatican City, an independent city-state and the headquarters of the Roman Catholic

### ConverseStream for streaming invocations

We can also use the Converse API for streaming invocations. In this case we rely on the ConverseStream action.

In [19]:
def invoke_bedrock_model_stream(client, id, prompt, max_tokens=2000, temperature=0, top_p=0.9):
    response = ""
    response = client.converse_stream(
        modelId=id,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "text": prompt
                    }
                ]
            }
        ],
        inferenceConfig={
            "temperature": temperature,
            "maxTokens": max_tokens,
            "topP": top_p
        }
    )
    # Extract and print the response text in real-time.
    for event in response['stream']:
        if 'contentBlockDelta' in event:
            chunk = event['contentBlockDelta']
            sys.stdout.write(chunk['delta']['text'])
            sys.stdout.flush()
    return

In [20]:
prompt = ("What is the capital of Italy?")
print(f'Prompt: {prompt}\n')

for i in MODEL_IDS:
    print(f'\n\nModel: {i}')
    invoke_bedrock_model_stream(bedrock, i, prompt)

Prompt: What is the capital of Italy?



Model: amazon.titan-text-express-v1
The capital of Italy is Rome. It is the fourth most populous city in the European Union.

Model: amazon.titan-text-lite-v1
The capital of Italy is Rome. Rome is the capital of both the country of Italy and the city of Rome.

Model: anthropic.claude-3-5-sonnet-20240620-v1:0
The capital of Italy is Rome.

Rome, also known as Roma in Italian, is not only the capital city but also the largest city in Italy. It has a rich history spanning over 2,500 years and is often referred to as the "Eternal City." Some key facts about Rome include:

1. It's located in the central-western part of the Italian peninsula.

2. It serves as the seat of the Italian government and is home to many important political and cultural institutions.

3. Vatican City, an independent city-state and the headquarters of the Roman Catholic Church, is located entirely within Rome.

4. Rome is renowned for its ancient ruins, art, architecture, and 

### Conversation with model specific parameters using Converse API

In this example we will call the Converse operation with the Anthropic Claude 3.5 Sonnet model. We will send the input text, inference parameters, and additional parameters that are unique to the model. We will start a conversation by asking the model to create a list of songs, then continues the conversation by asking that the songs are by artists from the United Kingdom.

In [21]:
def generate_conversation(bedrock_client,
                          model_id,
                          system_prompts,
                          messages):
    """
    Sends messages to a model.
    Args:
        bedrock_client: The Boto3 Bedrock runtime client.
        model_id (str): The model ID to use.
        system_prompts (JSON) : The system prompts for the model to use.
        messages (JSON) : The messages to send to the model.

    Returns:
        response (JSON): The conversation that the model generated.

    """

    print(f'Generating message with model {model_id}')

    # Inference parameters to use.
    temperature = 0.5
    top_k = 200

    # Base inference parameters to use.
    inference_config = {"temperature": temperature}
    # Additional inference parameters to use.
    additional_model_fields = {"top_k": top_k}

    # Send the message.
    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages,
        system=system_prompts,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields
    )

    # Log token usage.
    token_usage = response['usage']
    print(f"Input tokens: {token_usage['inputTokens']}")
    print(f"Output tokens: {token_usage['outputTokens']}")
    print(f"Total tokens: {token_usage['totalTokens']}")
    print(f"Stop reason: {response['stopReason']}")

    return response

In [22]:
from botocore.exceptions import ClientError

model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"

# Setup the system prompts and messages to send to the model.
system_prompts = [{"text": "You are an app that creates playlists for a radio station that plays rock and pop music."
                    "Only return song names and the artist."}]
message_1 = {
    "role": "user",
    "content": [{"text": "Create a list of 3 pop songs."}]
}
message_2 = {
    "role": "user",
    "content": [{"text": "Make sure the songs are by artists from the United Kingdom."}]
}
messages = []

try:

    bedrock_client = boto3.client(service_name='bedrock-runtime')

    # Start the conversation with the 1st message.
    messages.append(message_1)
    response = generate_conversation(
        bedrock_client, model_id, system_prompts, messages)

    # Add the response message to the conversation.
    output_message = response['output']['message']
    messages.append(output_message)

    # Continue the conversation with the 2nd message.
    messages.append(message_2)
    response = generate_conversation(
        bedrock_client, model_id, system_prompts, messages)

    output_message = response['output']['message']
    messages.append(output_message)

    # Show the complete conversation.
    for message in messages:
        print(f"Role: {message['role']}")
        for content in message['content']:
            print(f"Text: {content['text']}")
        print()

except ClientError as err:
    message = err.response['Error']['Message']
    print(f"A client error occured: {message}")

else:
    print(
        f"Finished generating text with model {model_id}.")

Generating message with model anthropic.claude-3-5-sonnet-20240620-v1:0
Input tokens: 45
Output tokens: 57
Total tokens: 102
Stop reason: end_turn
Generating message with model anthropic.claude-3-5-sonnet-20240620-v1:0
Input tokens: 117
Output tokens: 69
Total tokens: 186
Stop reason: end_turn
Role: user
Text: Create a list of 3 pop songs.

Role: assistant
Text: Here's a list of 3 pop songs:

1. "Shape of You" - Ed Sheeran
2. "Bad Guy" - Billie Eilish
3. "Blinding Lights" - The Weeknd

Role: user
Text: Make sure the songs are by artists from the United Kingdom.

Role: assistant
Text: I apologize for the oversight. Here's a list of 3 pop songs by artists from the United Kingdom:

1. "Waterloo" - ABBA
2. "Someone You Loved" - Lewis Capaldi
3. "Don't Start Now" - Dua Lipa

Finished generating text with model anthropic.claude-3-5-sonnet-20240620-v1:0.


### Conversation with Image using Converse API


In this example we will send an image as part of a message and requests that the model describe the image. The example uses Converse operation and the Anthropic Claude 3.5 Sonnet model.

In [25]:
def image_conversation(bedrock_client,
                          model_id,
                          input_text,
                          input_image):
    """
    Sends a message to a model.
    Args:
        bedrock_client: The Boto3 Bedrock runtime client.
        model_id (str): The model ID to use.
        input text : The input message.
        input_image : The input image.

    Returns:
        response (JSON): The conversation that the model generated.

    """

    print(f"Generating message with model {model_id}")

    # Message to send.

    with open(input_image, "rb") as f:
        image = f.read()

    message = {
        "role": "user",
        "content": [
            {
                "text": input_text
            },
            {
                    "image": {
                        "format": 'jpeg',
                        "source": {
                            "bytes": image
                        }
                    }
            }
        ]
    }

    messages = [message]

    # Send the message.
    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages
    )

    return response

In [26]:
model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
input_text = "What's in this image?"
input_image = "assets/sample_image.jpg"

try:

    bedrock_client = boto3.client(service_name="bedrock-runtime")

    response = image_conversation(
        bedrock_client, model_id, input_text, input_image)

    output_message = response['output']['message']

    print(f"Role: {output_message['role']}")

    for content in output_message['content']:
        print(f"Text: {content['text']}")

    token_usage = response['usage']
    print(f"Input tokens:  {token_usage['inputTokens']}")
    print(f"Output tokens:  {token_usage['outputTokens']}")
    print(f"Total tokens:  {token_usage['totalTokens']}")
    print(f"Stop reason: {response['stopReason']}")

except ClientError as err:
    message = err.response['Error']['Message']
    logger.error("A client error occurred: %s", message)
    print(f"A client error occured: {message}")

else:
    print(
        f"Finished generating text with model {model_id}.")

Generating message with model anthropic.claude-3-5-sonnet-20240620-v1:0
Role: assistant
Text: This image shows a group of nine dogs sitting together in a grassy field. The dogs are of various breeds and colors, including what appear to be Border Collies, a Belgian Malinois, and some mixed breeds. They're arranged in a line, facing the camera, with a beautiful green meadow and colorful flowers in the background. 

Some of the dogs have their tongues out, giving them a happy, friendly appearance. The dogs range in color from black, brown, and white, to combinations of these colors. Their ears are perked up, and they all seem alert and well-behaved, sitting still for the photo.

The setting looks like a lovely summer day, with the field in full bloom. It's a charming and heartwarming image that showcases the diversity and beauty of different dog breeds.
Input tokens:  1048
Output tokens:  177
Total tokens:  1225
Stop reason: end_turn
Finished generating text with model anthropic.claude-3-

### Conversation with Document using Converse API


In this example, we will send a document as part of a message and requests that the model describe the contents of the document. The example uses Converse operation and the Meta Llama 3.1 8B Instruct Model.

In [40]:
def document_conversation(bedrock_client,
                     model_id,
                     input_text,
                     input_document):
    """
    Sends a message to a model.
    Args:
        bedrock_client: The Boto3 Bedrock runtime client.
        model_id (str): The model ID to use.
        input text : The input message.
        input_document : The input document.

    Returns:
        response (JSON): The conversation that the model generated.

    """

    print(f"Generating message with model {model_id}")

    # Message to send.
    
    with open(input_document, "rb") as f:
        doc_bytes = f.read()

    message = {
        "role": "user",
        "content": [
            {
                "text": input_text
            },
            {
                "document": {
                    "name": "MyDocument",
                    "format": "pdf",
                    "source": {
                        "bytes": doc_bytes
                    }
                }
            }
        ]
    }

    messages = [message]

    # Send the message.
    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages
    )

    return response

In [50]:
model_id = "meta.llama3-1-8b-instruct-v1:0" 
input_text = "What's in this document?"
input_document = 'assets/2022-Shareholder-Letter.pdf'

try:

    bedrock_client = boto3.client(service_name="bedrock-runtime")

    response = document_conversation(
        bedrock_client, model_id, input_text, input_document)

    output_message = response['output']['message']

    print(f"Role: {output_message['role']}")

    for content in output_message['content']:
        print(f"Text: {content['text']}")

    token_usage = response['usage']
    print(f"Input tokens:  {token_usage['inputTokens']}")
    print(f"Output tokens:  {token_usage['outputTokens']}")
    print(f"Total tokens:  {token_usage['totalTokens']}")
    print(f"Stop reason: {response['stopReason']}")

except ClientError as err:
    message = err.response['Error']['Message']
    print(f"A client error occured: {message}")

else:
    print(
        f"Finished generating text with model {model_id}.")

Generating message with model meta.llama3-1-8b-instruct-v1:0
Role: assistant
Text: 

This document is a letter to Amazon shareholders from the company's CEO, Andy Jassy, dated 2023. The letter discusses the company's performance, strategy, and future prospects. Here are some of the key points mentioned in the letter:

1. **Amazon's performance**: The company's consumer business generated $434 billion in revenue in 2022, while its AWS business generated $80 billion in revenue.
2. **Market opportunity**: The letter highlights the vast market opportunity in e-commerce and cloud computing, with 80% of global retail still residing in physical stores and 90% of global IT spending still on-premises.
3. **Strategy**: Amazon's strategy is to focus on customer obsession, long-term market leadership, and relentless innovation.
4. **Investments**: The company is investing heavily in areas such as artificial intelligence, machine learning, and cloud computing.
5. **Future prospects**: The letter ex

## Next steps

Now that we have seen the Converse API allow us to easily run the invocations with the same syntax across all the models, you can learn


- How to do [function calling with the Converse API](../../agents/function-calling/function_calling_with_converse/function_calling_with_converse.ipynb)
- How to work with [Converse API and Guardrails for Amazon Bedrock](../../responsible_ai/) **Link To be updated**

